In [18]:
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

### Load Vocab

In [19]:
vocab = {}
idToWord = {}
with open('vocab.txt') as f:
    words = f.read().splitlines()
    for wordIndex in words:
        word, index = wordIndex.split(' : ')
        vocab[word] = index
        idToWord[int(index)] = word

### Create input squences

In [20]:
look_back_len = 5 + 1
sequences = []
vocabulary_size = len(vocab)

with open('int-seq.txt') as f:
    files = f.read().splitlines()
    for file in files:
        numbers = list(map(int, file.split(',')[:-1]))
        #print(numbers)
        for i in range(look_back_len, len(numbers)):
            seq = numbers[i-look_back_len:i]
            sequences.append(seq)
    #print(sequences)

    n_sequences = np.empty([len(sequences), look_back_len], dtype='int32')
    for i in range(len(sequences)):
        n_sequences[i] = sequences[i]

n_sequences

array([[ 2, 13, 17,  9,  0,  2],
       [13, 17,  9,  0,  2, 18],
       [17,  9,  0,  2, 18,  7],
       [ 9,  0,  2, 18,  7,  1],
       [ 0,  2, 18,  7,  1,  4],
       [ 2, 18,  7,  1,  4, 19],
       [18,  7,  1,  4, 19,  8],
       [ 7,  1,  4, 19,  8, 20],
       [ 1,  4, 19,  8, 20, 21],
       [ 4, 19,  8, 20, 21,  0],
       [19,  8, 20, 21,  0, 22],
       [ 8, 20, 21,  0, 22,  3],
       [20, 21,  0, 22,  3,  2],
       [21,  0, 22,  3,  2,  1],
       [ 0, 22,  3,  2,  1,  4],
       [22,  3,  2,  1,  4,  9],
       [ 3,  2,  1,  4,  9,  0],
       [ 2,  1,  4,  9,  0,  0],
       [ 1,  4,  9,  0,  0,  2],
       [ 4,  9,  0,  0,  2, 23],
       [ 9,  0,  0,  2, 23,  8],
       [ 0,  0,  2, 23,  8,  1],
       [ 0,  2, 23,  8,  1, 10],
       [ 2, 23,  8,  1, 10, 14],
       [23,  8,  1, 10, 14,  1],
       [ 8,  1, 10, 14,  1,  4],
       [ 1, 10, 14,  1,  4,  5],
       [10, 14,  1,  4,  5,  0],
       [14,  1,  4,  5,  0,  2],
       [ 1,  4,  5,  0,  2,  3],
       [ 4

### Convert output to one hot encoded vector

In [21]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]

train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
print(train_targets[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Train Model

In [22]:
model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
model.save("mymodel.h5")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 5)              185       
_________________________________________________________________
lstm_5 (LSTM)                (None, 5, 50)             11200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 37)                1887      
Total params: 36,022
Trainable params: 36,022
Non-trainable params: 0
_________________________________________________________________
None


c:\Miniconda\envs\azureml_py36_tensorflow\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
111/111 [==============================] - 1s 6ms/step - loss: 3.6092 - accuracy: 0.0721
Epoch 2/500
111/111 [==============================] - 0s 180us/step - loss: 3.6011 - accuracy: 0.1441
Epoch 3/500
111/111 [==============================] - 0s 234us/step - loss: 3.5901 - accuracy: 0.1532
Epoch 4/500
111/111 [==============================] - 0s 216us/step - loss: 3.5744 - accuracy: 0.1532
Epoch 5/500
111/111 [==============================] - 0s 207us/step - loss: 3.5526 - accuracy: 0.1532
Epoch 6/500
111/111 [==============================] - 0s 234us/step - loss: 3.5182 - accuracy: 0.1532
Epoch 7/500
111/111 [==============================] - 0s 216us/step - loss: 3.4632 - accuracy: 0.1532
Epoch 8/500
111/111 [==============================] - 0s 189us/step - loss: 3.3763 - accuracy: 0.1532
Epoch 9/500
111/111 [==============================] - 0s 198us/step - loss: 3.2387 - accuracy: 0.1712
Epoch 10/500
111/111 [==============================] - 0s 207us/step - los

111/111 [==============================] - 0s 189us/step - loss: 2.4173 - accuracy: 0.2523
Epoch 80/500
111/111 [==============================] - 0s 180us/step - loss: 2.3996 - accuracy: 0.2613
Epoch 81/500
111/111 [==============================] - 0s 198us/step - loss: 2.3891 - accuracy: 0.2793
Epoch 82/500
111/111 [==============================] - 0s 198us/step - loss: 2.3784 - accuracy: 0.2703
Epoch 83/500
111/111 [==============================] - 0s 189us/step - loss: 2.3631 - accuracy: 0.2703
Epoch 84/500
111/111 [==============================] - 0s 216us/step - loss: 2.3502 - accuracy: 0.2793
Epoch 85/500
111/111 [==============================] - 0s 189us/step - loss: 2.3424 - accuracy: 0.2793
Epoch 86/500
111/111 [==============================] - ETA: 0s - loss: 1.9444 - accuracy: 0.43 - 0s 180us/step - loss: 2.3329 - accuracy: 0.2703
Epoch 87/500
111/111 [==============================] - 0s 198us/step - loss: 2.3282 - accuracy: 0.2703
Epoch 88/500
111/111 [=============

111/111 [==============================] - 0s 180us/step - loss: 1.6497 - accuracy: 0.3784
Epoch 157/500
111/111 [==============================] - 0s 171us/step - loss: 1.6352 - accuracy: 0.3964
Epoch 158/500
111/111 [==============================] - 0s 180us/step - loss: 1.6389 - accuracy: 0.4054
Epoch 159/500
111/111 [==============================] - 0s 189us/step - loss: 1.6330 - accuracy: 0.4324
Epoch 160/500
111/111 [==============================] - 0s 207us/step - loss: 1.6296 - accuracy: 0.4324
Epoch 161/500
111/111 [==============================] - 0s 180us/step - loss: 1.6146 - accuracy: 0.4144
Epoch 162/500
111/111 [==============================] - 0s 171us/step - loss: 1.6115 - accuracy: 0.4324
Epoch 163/500
111/111 [==============================] - 0s 180us/step - loss: 1.5957 - accuracy: 0.4595
Epoch 164/500
111/111 [==============================] - 0s 171us/step - loss: 1.5995 - accuracy: 0.4324
Epoch 165/500
111/111 [==============================] - 0s 171us/ste

111/111 [==============================] - 0s 180us/step - loss: 1.1570 - accuracy: 0.6036
Epoch 234/500
111/111 [==============================] - 0s 198us/step - loss: 1.1515 - accuracy: 0.5856
Epoch 235/500
111/111 [==============================] - 0s 198us/step - loss: 1.1384 - accuracy: 0.6036
Epoch 236/500
111/111 [==============================] - 0s 180us/step - loss: 1.1364 - accuracy: 0.6216
Epoch 237/500
111/111 [==============================] - 0s 198us/step - loss: 1.1238 - accuracy: 0.6306
Epoch 238/500
111/111 [==============================] - 0s 207us/step - loss: 1.1293 - accuracy: 0.6126
Epoch 239/500
111/111 [==============================] - 0s 207us/step - loss: 1.1189 - accuracy: 0.6216
Epoch 240/500
111/111 [==============================] - 0s 189us/step - loss: 1.1148 - accuracy: 0.6216
Epoch 241/500
111/111 [==============================] - 0s 171us/step - loss: 1.1130 - accuracy: 0.5856
Epoch 242/500
111/111 [==============================] - 0s 189us/ste

111/111 [==============================] - 0s 180us/step - loss: 0.7309 - accuracy: 0.7838
Epoch 312/500
111/111 [==============================] - 0s 207us/step - loss: 0.7282 - accuracy: 0.7477
Epoch 313/500
111/111 [==============================] - 0s 189us/step - loss: 0.7182 - accuracy: 0.7658
Epoch 314/500
111/111 [==============================] - 0s 189us/step - loss: 0.7159 - accuracy: 0.7748
Epoch 315/500
111/111 [==============================] - 0s 180us/step - loss: 0.7111 - accuracy: 0.8018
Epoch 316/500
111/111 [==============================] - 0s 171us/step - loss: 0.7044 - accuracy: 0.7838
Epoch 317/500
111/111 [==============================] - 0s 180us/step - loss: 0.7029 - accuracy: 0.7928
Epoch 318/500
111/111 [==============================] - 0s 180us/step - loss: 0.7080 - accuracy: 0.7568
Epoch 319/500
111/111 [==============================] - 0s 180us/step - loss: 0.6940 - accuracy: 0.7568
Epoch 320/500
111/111 [==============================] - ETA: 0s - lo

111/111 [==============================] - 0s 171us/step - loss: 0.4728 - accuracy: 0.8468
Epoch 389/500
111/111 [==============================] - 0s 225us/step - loss: 0.4681 - accuracy: 0.8739
Epoch 390/500
111/111 [==============================] - 0s 198us/step - loss: 0.4678 - accuracy: 0.8649
Epoch 391/500
111/111 [==============================] - 0s 189us/step - loss: 0.4734 - accuracy: 0.8739
Epoch 392/500
111/111 [==============================] - 0s 171us/step - loss: 0.4650 - accuracy: 0.8649
Epoch 393/500
111/111 [==============================] - 0s 189us/step - loss: 0.4863 - accuracy: 0.8378
Epoch 394/500
111/111 [==============================] - 0s 180us/step - loss: 0.4686 - accuracy: 0.8649
Epoch 395/500
111/111 [==============================] - 0s 171us/step - loss: 0.4776 - accuracy: 0.8559
Epoch 396/500
111/111 [==============================] - 0s 171us/step - loss: 0.4673 - accuracy: 0.8649
Epoch 397/500
111/111 [==============================] - 0s 189us/ste

111/111 [==============================] - 0s 189us/step - loss: 0.3004 - accuracy: 0.9550
Epoch 467/500
111/111 [==============================] - 0s 189us/step - loss: 0.2951 - accuracy: 0.9640
Epoch 468/500
111/111 [==============================] - 0s 171us/step - loss: 0.2946 - accuracy: 0.9279
Epoch 469/500
111/111 [==============================] - 0s 198us/step - loss: 0.2908 - accuracy: 0.9550
Epoch 470/500
111/111 [==============================] - 0s 198us/step - loss: 0.2906 - accuracy: 0.9730
Epoch 471/500
111/111 [==============================] - 0s 189us/step - loss: 0.2829 - accuracy: 0.9640
Epoch 472/500
111/111 [==============================] - 0s 189us/step - loss: 0.2934 - accuracy: 0.9550
Epoch 473/500
111/111 [==============================] - 0s 189us/step - loss: 0.2870 - accuracy: 0.9550
Epoch 474/500
111/111 [==============================] - 0s 207us/step - loss: 0.2922 - accuracy: 0.9640
Epoch 475/500
111/111 [==============================] - 0s 198us/ste

### Test Model

In [23]:
from keras.preprocessing.sequence import pad_sequences

# sample python input
"""
num = 46
print
"""
encoded_text = [2,13,35,5]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre', valu)
print(encoded_text, pad_encoded)

print("Top 3 Suggestions:")
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    print(idToWord[i])

[2, 13, 35, 5] [[ 0  2 13 35  5]]
Top 3 Suggestions:
else
(
print
